In [62]:
import numpy as np
from scipy.integrate import quad
from IPython.display import clear_output

def toK(c):
    return (c + 273.15)

def toPa(bar):
    return (bar * 1e5)

def toBar(Pa):
    return (Pa / 1e5)
    
R = 8.314462618 # J/(K·mol)
R_Bar =  0.00008314462618 # bar·m³/(K·mol)
Tpad = 298.15 # temperatura padrão 25 Celcius
Ppad = 1 # Bar

class Comp: # Compostos : Classe que abriga as propriedades e funções
    def __init__(self, name, omega, Pc, Tb, Tc, NCParams, Ant = None, Hf = 0): # NCParams: A, B, C, D cp do V.Ness; Ant: A, B, C 
        self.name = name
        self.omega = omega
        self.Pc = Pc
        self.Tb = Tb
        self.Tc = Tc
        self.NCParams = NCParams
        self.Ant = Ant
        self.Hf = Hf

    def Antoine(self, T): # NIST, T em K, P em Bar
        AntA = self.Ant[0]
        AntB = self.Ant[1]
        AntC = self.Ant[2]
        if T < self.Ant[3] or T > self.Ant[4]:
            print(f"{self.name} : Temperatura {T} é fora da faixa para antoine")
        return 10 ** (AntA - (AntB / (T + AntC)))

    def TAntoine(self, P): # Eq antoine para temperatura, P em Bar
        Ant = self.Ant
        T = (Ant[1] / (Ant[0] - np.log10(P))) - Ant[2]
        return T

    def Estado(self, P, T): # Retorna o estado do composto
        if T > self.Tc and P > self.Pc:
            print(f"{self.name} : Fluído supercrítico")
            return "gas"
        elif T > self.Tc or self.Ant == None:
            return "gas"
        elif T < self.Tc and P > self.Pc:
            return "liquid"
        elif T < self.Tc and P < self.Pc:
            if P == Ppad:
                if T > self.Tb:
                    return "gas"
                else:
                    return "liquid"
            if  P < self.Antoine(T):
                return "vapour"
            elif P == self.Antoine(T):
                print(f"{self.name} : FODEO")
            else:
                return "liquid"

    def Cp_ig(self, T): # Equação 142 V.Ness
        A = self.NCParams[0]
        B =  self.NCParams[1]
        C = self.NCParams[2]
        D =  self.NCParams[3]
        return R * (A + (B * T) + (C * (T ** 2)) + (D * (T ** (-2))))

    def Cp_L(self, T): # Cp de Rowlison Bondi
        return R * (1.45 + (0.45/(1-(T / self.Tc)) + 0.25 * self.omega * (17.11 + (25.2 * (1 - (T / self.Tc))**(1/3)) / (T / self.Tc) + (1.742)/(1 - (T / self.Tc)) ) ) ) + self.Cp_ig(T)
    
    def Hres(self, P, T): # Por Peng Robinson slide aula 2 integração
        P = toPa(P)
        k = 0.37464 + (0.154226 * self.omega) - (0.26992 * (self.omega ** 2))
        alphat = (1 + k * (1 - np.sqrt(T/self.Tc))) ** 2
        ac = 0.45724 * (((R ** 2) * (self.Tc ** 2)) / toPa(self.Pc))
        a = ac * alphat
        b = 0.07780 * R * (self.Tc / toPa(self.Pc))
        A = (a * P) / ((R ** 2) * (T ** 2.5))
        B =  (b * P) / (R * T)
        alpha = -1 + B
        beta = A - (3 * B ** 2 ) - 2 * B
        gamma = - (A * B) + (B ** 2) + (B ** 3) 
        Z = np.roots([1, alpha, beta, gamma]) # Resolução cúbica
        Z = Z[np.isreal(Z)] 
        Z = max(Z)
        Tdadt = - ac * k * alphat * (T / self.Tc)
        h = (b * P) / (Z * R * T)
        return R * T * (  (Z - 1) - ((a - Tdadt)/(2 * np.sqrt(2) * b * R * T)) * np.log((1 + h * (1 + np.sqrt(2)))/(1 + h * (1 - np.sqrt(2))))  )

    def Hvap(self): # Por Riedel p147 V.Ness
        return R * self.Tb * ((1.092 * (np.log(self.Pc) - 1.013)) / (0.930 - (self.Tb / self.Tc))) # P em bar mesmo

    def Hmolar(self, P, T): # Calcula a entalpia molar
        match self.Estado(Ppad, Tpad): # estado inicial
            case "gas" | "vapour":
                match self.Estado(Ppad, T): # estado final
                    case "gas" | "vapour":
                        print("caminho gás - gás")
                        res = self.Hres(P, T)
                        ig = quad(self.Cp_ig, Tpad, T)[0]
                        H = res + ig
                        print(f"{self.name} : Hig = {ig:.3f} J/mol, HRes {res:.3f} J/mol, Htotal {H:.3f} J/mol")
                        return H
                    case "liquid":
                        print("caminho gás - líquido")
                        Tvap = self.TAntoine(Ppad)
                        res = self.Hres(P, T)
                        ig = quad(self.Cp_ig, Tpad, T)[0]
                        HGas = res + ig
                        Hv = self.Hvap()
                        Hl = quad(self.Cp_L, Tvap, T)[0]
                        H = HGas - Hv + Hl # Menos Hvaporização já que é o processo inverso, condensação
                        print(f"{self.name} :T Antoine: {Tvap}, Hig = {ig:.3f} J/mol, HRes {res:.3f} J/mol, HGas {HGas:.3f} J/mol, Hv {Hv:.3f} J/mol, Hl {Hl:.3f} J/mol, Htotal {H:.3f} J/mol")
                        return H
            case "liquid":
                match self.Estado(Ppad, T): #estado final
                    case "gas" | "vapour":
                        print("caminho líquido - gás")
                        Tvap = self.TAntoine(Ppad)
                        Hl = quad(self.Cp_L, Tpad, Tvap)[0]
                        Hv = self.Hvap()
                        res = self.Hres(P, T)
                        ig = quad(self.Cp_ig, Tpad, T)[0]
                        HGas = res + ig
                        H = Hl + Hv + HGas
                        print(f"{self.name} : T Antoine: {Tvap}, Hig = {ig:.3f} J/mol, HRes {res:.3f} J/mol, HGas {HGas:.3f} J/mol, Hv {Hv:.3f} J/mol, Hl {Hl:.3f} J/mol, Htotal {H:.3f} J/mol")
                        return H
                    case "liquid":
                        print("caminho líquido - líquido")
                        H = quad(self.Cp_L, Tpad, T)[0]
                        print(f"{self.name} : Htotal e Hl {H:.3f} J/mol")
                        return H

# Componentes

C2H4O = Comp( # 
        "Oxido de etileno",
        0.201, # omega Perry
        toBar(7.26e6), # Pc Perry
        toK(10.4), # Tb
        469.15, # Tc Perry
        [-0.385, # Cp A V.Ness
        23.463e-3, # B
        -9.296e-6, # C
        0], # D
        [5.84696, 2022.83, 62.656, 273.4, 304.9], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -52.630) # Hf J/mol  V. Ness

O2 = Comp( # 
        "Oxigênio",
        0.020, # omega Perry
        toBar(5.02e6), # Pc Perry
        toK(-182.953), # Tb
        154.58, # Tc Perry
        [3.639, # Cp A V.Ness
        0.506e-3, # B
        0, # C
        -0.227e5], # D
        # [3.9523, 340.024, --4.144, 54.36, 154.33], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 0) # Hf J/mol 

N2 = Comp( #
        "Nitrogênio",
        0.037, # omega Perry
        toBar(3.39e6), # Pc Perry
        toK(-195.798), # Tb
        126.2, # Tc Perry
        [3.280, # Cp A V.Ness
        0.593e-3, # B
        0, # C
        0.040e5], # D
        # [3.7362, 264.651, -6.788, 63.14, 126], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 0) # Hf J/mol 

C2H4 = Comp( # 
        "Etileno",
        0.086, # omega Perry
        toBar(5.03e6), # Pc Perry
        toK(-103.8), # Tb
        282.34, # Tc Perry
        [1.424, # Cp A V.Ness
        14.394e-3, # B
        -4.392e-6, # C
        0], # D
        [13.871, 2854.7, -36.23, toK(-169.16), toK(76.52)], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = 52.510) # Hf J/mol  V. Ness

CO2 = Comp( #
        "Dioxido de carbono",
        0.224, # omega Perry
        toBar(7.39e6), # Pc Perry
        toK(-78.464), # Tb
        304.21, # Tc Perry
        [5.457, # Cp A V.Ness 
        1.045e-3, # B 
        0, # C 
        -1.157e5], # D 
        # [6.81228, 1301.679, -3.494, 154.26, 195.89]   , # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -393.509) # Hf J/mol V. Ness 
    
CH3CHO = Comp( # 
        "Acetaldeído",
        0.292, # omega Perry
        toBar(5.57e6), # Pc Perry
        toK(20.6), # Tb Haynes
        466, # Tc Perry
        [-0.385, # Cp A V.Ness
        23.463e-3, # B
        -9.296e-6, # C
        0], # D
        [3.68639, 822.894, -69.899, 293.4, 377.5], # Antoine A, B, C, D, Tmin, Tmax 
        Hf = -166.190) # J/mol V. Ness

H2O = Comp( # 
        "Água",
        0.343, # omega Perry
        toBar(21.94e6), # Pc Perry
        toK(99.974), # Tb Haynes
        647.13, # Tc Perry
        [3.470, # Cp A V.Ness
        1.450e-3, # B
        0, # C
        0.121e5], # D
        [4.6543, 1435.264, -64.848, 255.9, 373], # Antoine A, B, C, D, Tmin, Tmax
        Hf = -285.830) # Líquido J/mol Hf V. Ness


class Corrente: # Corrente : Classe que abriga uma corrente toda
    def __init__(self, name, P, T, C2H4O = 0, O2 = 0, N2 = 0, C2H4 = 0, CO2 = 0, H2O = 0, CH3CHO = 0): # pressão, temperatura e vazões molares kmol / h
        self.name =  name
        self.P = P
        self.T = T
        self.C2H4O = C2H4O
        self.O2 = O2
        self.N2 = N2
        self.C2H4 = C2H4
        self.CO2 = CO2
        self.H2O = H2O
        self.CH3CHO = CH3CHO
    def Hcorr(self):
        H = 0
        if self.C2H4O != 0:
            H += self.C2H4O * C2H4O.Hmolar(self.P, self.T)
        if self.O2 != 0:
            H += self.O2 * O2.Hmolar(self.P, self.T)
        if self.N2 != 0:
            H += self.N2 * N2.Hmolar(self.P, self.T)
        if self.C2H4 != 0:
            H += self.C2H4 * C2H4.Hmolar(self.P, self.T)
        if self.CO2 != 0:
            H += self.CO2 * CO2.Hmolar(self.P, self.T)
        if self.H2O != 0:
            H += self.H2O * H2O.Hmolar(self.P, self.T)
        if self.CH3CHO != 0:
            H += self.CH3CHO * CH3CHO.Hmolar(self.P, self.T)
            
        print(f"\n{self.name} : {H:.3f} kJ/h\n")
        return H

In [63]:
# TC-01
C3 = Corrente("Corrente 3, Entrada TC-01", 22, toK(30), 0.001828269, 1399.380465, 5266.194335, 2798.76093, 0.183260069, 0.021612713, 0)
C4a = Corrente("Corrente 4a, Saída TC-01", 20, toK(140), 0.001828269, 1399.380465, 5266.194335, 2798.76093, 0.183260069, 0.021612713, 0)
C5 = Corrente("Corrente 5, Entrada TC-01", 19, toK(240), 226.3148763, 1102.180042, 5266.194335, 2508.809297, 121.9629456, 121.8012983, 2.748741474)
C5a = Corrente("Corrente 5a, Saída TC-01", 19, toK(150), 226.3148763, 1102.180042, 5266.194335, 2508.809297, 121.9629456, 121.8012983, 2.748741474)

dH34 = C4a.Hcorr() - C3.Hcorr()
dH5 = C5a.Hcorr() - C5.Hcorr()
while dH34 + dH5 > 1:
    C4a.T -= 0.1
    C5a.T = C4a.T + 10
    dH34 = C4a.Hcorr() - C3.Hcorr()
    dH5 = C5a.Hcorr() - C5.Hcorr()
C4a.T = toK(130) # round(C4a.T, 0)
C5a.T = C4a.T + 10
clear_output(wait=False)
dH34 = C4a.Hcorr() - C3.Hcorr()
dH5 = C5a.Hcorr() - C5.Hcorr()
print(f"C4a.T {C4a.T - 273.15}, Delta C3: {dH34:.3f}, Delta C5 {dH5:.3f} \n dif: {(dH34 + dH5):.3f}")


caminho gás - líquido
Oxido de etileno :T Antoine: 283.30668830298134, Hig = -5857.831 J/mol, HRes -2952.989+0.000j J/mol, HGas -8810.819+0.000j J/mol, Hv 25870.130 J/mol, Hl -12566.110 J/mol, Htotal -47247.059+0.000j J/mol
caminho gás - gás
Oxigênio : Hig = -4420.242 J/mol, HRes -354.183 J/mol, Htotal -4774.426 J/mol
caminho gás - gás
Nitrogênio : Hig = -4907.455 J/mol, HRes -325.948 J/mol, Htotal -5233.403 J/mol
caminho gás - líquido
Etileno :T Antoine: 242.0334748756398, Hig = -6003.021 J/mol, HRes -1323.755+0.000j J/mol, HGas -7326.776+0.000j J/mol, Hv 13527.681 J/mol, Hl -7445.672 J/mol, Htotal -28300.128+0.000j J/mol
caminho gás - gás
Dioxido de carbono : Hig = -3768.703 J/mol, HRes -1095.449+0.000j J/mol, Htotal -4864.152+0.000j J/mol
caminho líquido - líquido
Água : Htotal e Hl -19572.679 J/mol

Corrente 4a, Saída TC-01 : -113448049.768+0.000j kJ/h

caminho gás - gás
Oxido de etileno : Hig = 242.319 J/mol, HRes -1115.243 J/mol, Htotal -872.923 J/mol
caminho gás - gás
Oxigênio :

In [64]:
# F-01
# QF1 = C4b - C4a
C4b = Corrente("Corrente 4b, Saída Forno", 20, toK(200),  0.001828269, 1399.380465, 5266.194335, 2798.76093, 0.183260069, 0.021612713, 0)
QF1 = C4b.Hcorr() - C4a.Hcorr() 
print(f"F-01: {QF1} kJ/h")

caminho gás - gás
Oxido de etileno : Hig = 10559.488 J/mol, HRes -591.708 J/mol, Htotal 9967.780 J/mol
caminho gás - gás
Oxigênio : Hig = 5344.657 J/mol, HRes -45.025 J/mol, Htotal 5299.632 J/mol
caminho gás - gás
Nitrogênio : Hig = 5146.510 J/mol, HRes -27.401 J/mol, Htotal 5119.110 J/mol
caminho gás - gás
Etileno : Hig = 9182.167 J/mol, HRes -254.425 J/mol, Htotal 8927.742 J/mol
caminho gás - gás
Dioxido de carbono : Hig = 7333.126 J/mol, HRes -212.021 J/mol, Htotal 7121.106 J/mol
caminho líquido - gás
Água : T Antoine: 373.22176189760006, Hig = 5987.403 J/mol, HRes -412.499 J/mol, HGas 5574.904 J/mol, Hv 41965.052 J/mol, Hl 7187.802 J/mol, Htotal 54727.758 J/mol

Corrente 4b, Saída Forno : 59363549.174 kJ/h

caminho gás - líquido
Oxido de etileno :T Antoine: 283.30668830298134, Hig = -5857.831 J/mol, HRes -2952.989+0.000j J/mol, HGas -8810.819+0.000j J/mol, Hv 25870.130 J/mol, Hl -12566.110 J/mol, Htotal -47247.059+0.000j J/mol
caminho gás - gás
Oxigênio : Hig = -4420.242 J/mol, HRe

In [65]:
# R-01
# Reação lei de hess
R01 = (C5.C2H4O - C4b.C2H4O) * C2H4O.Hf + (C5.C2H4 - C4b.C2H4) * C2H4.Hf + (C5.CO2 - C4b.CO2) * CO2.Hf + (C5.H2O - C4b.H2O) * H2O.Hf + (C5.CH3CHO - C4b.CH3CHO) * CH3CHO.Hf
print(f"Reação: {R} kJ/h")
H45 = C5.Hcorr() - C4b.Hcorr()
print(f"Delta H, 4b - 5 : {H45} kJ/h")
Hr = R01 + H45
print(f"R-01: {Hr} kJ/h")

Reação: 8.314462618 kJ/h
caminho gás - gás
Oxido de etileno : Hig = 13527.345 J/mol, HRes -507.787 J/mol, Htotal 13019.558 J/mol
caminho gás - gás
Oxigênio : Hig = 6606.806 J/mol, HRes -34.092 J/mol, Htotal 6572.713 J/mol
caminho gás - gás
Nitrogênio : Hig = 6340.106 J/mol, HRes -17.391 J/mol, Htotal 6322.714 J/mol
caminho gás - gás
Etileno : Hig = 11661.105 J/mol, HRes -213.935 J/mol, Htotal 11447.170 J/mol
caminho gás - gás
Dioxido de carbono : Hig = 9160.915 J/mol, HRes -178.747 J/mol, Htotal 8982.168 J/mol
caminho líquido - gás
Água : T Antoine: 373.22176189760006, Hig = 7395.840 J/mol, HRes -355.226 J/mol, HGas 7040.614 J/mol, Hv 41965.052 J/mol, Hl 7187.802 J/mol, Htotal 56193.469 J/mol
caminho gás - gás
Acetaldeído : Hig = 13527.345 J/mol, HRes -645.272 J/mol, Htotal 12882.074 J/mol

Corrente 5, Entrada TC-01 : 80181581.045 kJ/h

caminho gás - gás
Oxido de etileno : Hig = 10559.488 J/mol, HRes -591.708 J/mol, Htotal 9967.780 J/mol
caminho gás - gás
Oxigênio : Hig = 5344.657 J/mo

In [66]:
# TC-02
# QTC2 = C5b - C5a
C5b = Corrente("Corrente 5a, Saída TC-02", 18, toK(50), 226.3148763, 1102.180042, 5266.194335, 2508.809297, 121.9629456, 121.8012983, 2.748741474)
QTC2 = C5b.Hcorr() - C5a.Hcorr() 
print(f"TC-02: {QTC2} kJ/h")

caminho gás - gás
Oxido de etileno : Hig = 1248.458 J/mol, HRes -854.335 J/mol, Htotal 394.124 J/mol
caminho gás - gás
Oxigênio : Hig = 740.108 J/mol, HRes -89.511 J/mol, Htotal 650.598 J/mol
caminho gás - gás
Nitrogênio : Hig = 728.707 J/mol, HRes -73.208 J/mol, Htotal 655.499 J/mol
caminho gás - gás
Etileno : Hig = 1137.299 J/mol, HRes -390.359 J/mol, Htotal 746.940 J/mol
caminho gás - gás
Dioxido de carbono : Hig = 952.165 J/mol, HRes -322.254 J/mol, Htotal 629.911 J/mol
caminho líquido - líquido
Água : Htotal e Hl 2440.003 J/mol
caminho gás - gás
Acetaldeído : Hig = 1248.458 J/mol, HRes -1083.923 J/mol, Htotal 164.536 J/mol

Corrente 5a, Saída TC-02 : 6506658.245 kJ/h

caminho gás - líquido
Oxido de etileno :T Antoine: 283.30668830298134, Hig = -5640.573 J/mol, HRes -2522.643+0.000j J/mol, HGas -8163.216+0.000j J/mol, Hv 25870.130 J/mol, Hl -11767.128 J/mol, Htotal -45800.474+0.000j J/mol
caminho gás - gás
Oxigênio : Hig = -4215.702 J/mol, HRes -307.579 J/mol, Htotal -4523.281 J/mo

In [67]:
# Dest-01
# Roult na corrente de fundo
C12 = Corrente("Corrente 12, Entrada", 5.8, H2O.TAntoine(5.8), C2H4O = 225.1780362, H2O = 65.83972599, CH3CHO = 2.748741474)
C15 = Corrente("Corrente 15, Topo", 5.4, C2H4O.TAntoine(5.8), C2H4O = 675.5341086, CH3CHO = 5.447855715)
C16 = Corrente("Corrente 16, Fundo", 5.6, H2O.TAntoine(5.6 * (65.83972599 /(65.83972599 + 0.932789569)) ), H2O = 65.83972599, CH3CHO = 0.932789569)
C15c = Corrente("Corrente 15, Refluxo", 5.4, C2H4O.TAntoine(5.4), C2H4O = 450.3560724, CH3CHO = 3.63190381)

# C15c - C15 = TC-06
TC06 = C15c.Hcorr() - C15.Hcorr() 

# C15 + C16 - C15 - C15c = Q, Q = TC-06 + TC-07
TC07 = C15.Hcorr() + C16.Hcorr() - C15.Hcorr() - C15c.Hcorr() - TC06 

print(f"TC-06: {TC06} kJ/h, TC-07: {TC07} kJ/h")

caminho gás - gás
Oxido de etileno : Hig = 1757.296 J/mol, HRes -255.494 J/mol, Htotal 1501.803 J/mol
caminho gás - gás
Acetaldeído : Hig = 1757.296 J/mol, HRes -327.321 J/mol, Htotal 1429.975 J/mol

Corrente 15, Refluxo : 681539.503 kJ/h

caminho gás - gás
Oxido de etileno : Hig = 1886.090 J/mol, HRes -253.256 J/mol, Htotal 1632.834 J/mol
caminho gás - gás
Acetaldeído : Hig = 1886.090 J/mol, HRes -324.455 J/mol, Htotal 1561.635 J/mol

Corrente 15, Topo : 1111542.686 kJ/h

caminho gás - gás
Oxido de etileno : Hig = 1886.090 J/mol, HRes -253.256 J/mol, Htotal 1632.834 J/mol
caminho gás - gás
Acetaldeído : Hig = 1886.090 J/mol, HRes -324.455 J/mol, Htotal 1561.635 J/mol

Corrente 15, Topo : 1111542.686 kJ/h

caminho líquido - gás
Água : T Antoine: 373.22176189760006, Hig = 4545.529 J/mol, HRes -130.720 J/mol, HGas 4414.809 J/mol, Hv 41965.052 J/mol, Hl 7187.802 J/mol, Htotal 53567.664 J/mol
caminho gás - gás
Acetaldeído : Hig = 7690.997 J/mol, HRes -245.913 J/mol, Htotal 7445.084 J/mol



In [68]:
# Utilidades
# Água de resfriamento
agua = H2O.Hmolar(Ppad, toK(40))

caminho líquido - líquido
Água : Htotal e Hl 1470.912 J/mol
